In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DATASET_PATH = "../data/Loan_Default.csv"

MAP_OPEN_CREDIT      = {'opc': 1, 'nopc': 0}
MAP_NEG_AMMO         = {'neg_amm': 1, 'not_neg': 0}
MAP_INTEREST_ONLY    = {'int_only': 1, 'not_int': 0}
MAP_LUMP_SUM_PAYMENT = {'lpsm': 1, 'not_lpsm': 0}
MAP_AGE              = {'<25': 0, '25-34': 1, '35-44': 2, '45-54': 3, '55-64': 4, '65-74': 5, '>74': 6 }
MAP_REGION           = {'south':0, 'North': 1, 'central': 2, 'North-East': 3}
MAP_BS_OR_COMM       = {'nob/c': 0, 'b/c': 1}
MAP_OCC_TYPE         = {'pr': 0, 'sr': 1, 'ir': 2}
MAP_SECURED_BY       = {'home': 0, 'land': 1}

LOW_QUANTILE = 0.10
HIGH_QUANTILE = 0.90
IRQ_COEFF = 3

# Categorical encoding

In [3]:
ds = pd.read_csv(DATASET_PATH)
important_df =  ds[['loan_amount', 'rate_of_interest', 'Interest_rate_spread', 'Upfront_charges',
                    'term', 'property_value', 'LTV', 'Credit_Score', 'income', 'dtir1',
                    'open_credit', 'Neg_ammortization', 'interest_only', 'lump_sum_payment',
                    'age', 'Region', 'business_or_commercial', 'occupancy_type', 'Secured_by',
                    'Status']]

important_df.loc[:, 'open_credit']            = important_df['open_credit'].map(MAP_OPEN_CREDIT)
important_df.loc[:, 'Neg_ammortization']      = important_df['Neg_ammortization'].map(MAP_NEG_AMMO)
important_df.loc[:, 'interest_only']          = important_df['interest_only'].map(MAP_INTEREST_ONLY)
important_df.loc[:, 'lump_sum_payment']       = important_df['lump_sum_payment'].map(MAP_LUMP_SUM_PAYMENT)
important_df.loc[:, 'age']                    = important_df['age'].map(MAP_AGE)
important_df.loc[:, 'Region']                 = important_df['Region'].map(MAP_REGION)
important_df.loc[:, 'business_or_commercial'] = important_df['business_or_commercial'].map(MAP_BS_OR_COMM)
important_df.loc[:, 'occupancy_type']         = important_df['occupancy_type'].map(MAP_OCC_TYPE)
important_df.loc[:, 'Secured_by']             = important_df['Secured_by'].map(MAP_SECURED_BY)

important_df = important_df.apply(lambda col: pd.to_numeric(col, errors='coerce') if col.dtype == 'object' else col)

important_df.drop(columns=['LTV', 'dtir1', 'Interest_rate_spread'], inplace=True)

In [4]:
important_df.head(5)

,loan_amount,rate_of_interest,Upfront_charges,term,property_value,Credit_Score,income,open_credit,Neg_ammortization,interest_only,lump_sum_payment,age,Region,business_or_commercial,occupancy_type,Secured_by,Status
0,116500,NaN,NaN,360.0,118000.0,758,1740.0,0,0.0,0,0,1.0,0,0,0,0,1
1,206500,NaN,NaN,360.0,NaN,552,4980.0,0,0.0,0,1,4.0,1,1,0,0,1
2,406500,4.56,595.0,360.0,508000.0,834,9480.0,0,1.0,0,0,2.0,0,0,0,0,0
3,456500,4.25,NaN,360.0,658000.0,587,11880.0,0,0.0,0,0,3.0,1,0,0,0,0
4,696500,4.00,0.0,360.0,758000.0,602,10440.0,0,0.0,0,0,1.0,1,0,0,0,0


# Handling Nan values

In [5]:
missing = list()
for x in important_df.columns:
    if important_df[x].isnull().sum() != 0:
        print(f"{x:<30}{important_df[x].isnull().sum():<10}{(important_df[x].isnull().sum() / important_df.shape[0])*100}%")
        missing.append(x)

rate_of_interest              36439     24.509988565278807%
Upfront_charges               39642     26.664424564471652%
term                          41        0.027577856998722002%
property_value                15098     10.15537768211475%
income                        9150      6.154570525324544%
Neg_ammortization             121       0.08138830967915517%
age                           200       0.13452613170108293%


In [6]:
for col in missing:
    if col == 'Neg_ammortization' or col == 'age':
        important_df[col].fillna(important_df[col].mode()[0], inplace=True)
    else:
        important_df[col].fillna(important_df[col].median(), inplace=True)

print(important_df.isnull().sum())

loan_amount               0
rate_of_interest          0
Upfront_charges           0
term                      0
property_value            0
Credit_Score              0
income                    0
open_credit               0
Neg_ammortization         0
interest_only             0
lump_sum_payment          0
age                       0
Region                    0
business_or_commercial    0
occupancy_type            0
Secured_by                0
Status                    0
dtype: int64


/tmp/ipykernel_7758/1491325868.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  important_df[col].fillna(important_df[col].median(), inplace=True)
/tmp/ipykernel_7758/1491325868.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [7]:
columns_to_convert = ['Neg_ammortization', 'age']
important_df[columns_to_convert] = important_df[columns_to_convert].astype(int)

print(important_df.dtypes)

loan_amount                 int64
rate_of_interest          float64
Upfront_charges           float64
term                      float64
property_value            float64
Credit_Score                int64
income                    float64
open_credit                 int64
Neg_ammortization           int64
interest_only               int64
lump_sum_payment            int64
age                         int64
Region                      int64
business_or_commercial      int64
occupancy_type              int64
Secured_by                  int64
Status                      int64
dtype: object


In [8]:
important_df.head(5)

,loan_amount,rate_of_interest,Upfront_charges,term,property_value,Credit_Score,income,open_credit,Neg_ammortization,interest_only,lump_sum_payment,age,Region,business_or_commercial,occupancy_type,Secured_by,Status
0,116500,3.99,2596.45,360.0,118000.0,758,1740.0,0,0,0,0,1,0,0,0,0,1
1,206500,3.99,2596.45,360.0,418000.0,552,4980.0,0,0,0,1,4,1,1,0,0,1
2,406500,4.56,595.00,360.0,508000.0,834,9480.0,0,1,0,0,2,0,0,0,0,0
3,456500,4.25,2596.45,360.0,658000.0,587,11880.0,0,0,0,0,3,1,0,0,0,0
4,696500,4.00,0.00,360.0,758000.0,602,10440.0,0,0,0,0,1,1,0,0,0,0


# Handling Anomalies

In [9]:
def detect_anomalies(df, column):
    Q1 = df[column].quantile(LOW_QUANTILE)
    Q3 = df[column].quantile(HIGH_QUANTILE)
    IQR = Q3 - Q1
    lower_bound = Q1 - IRQ_COEFF * IQR
    upper_bound = Q3 + IRQ_COEFF * IQR
    anomalies = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return anomalies

numerical_columns = ['loan_amount', 'rate_of_interest', 'Upfront_charges', 'term', 'property_value', 'Credit_Score', 'income']
for col in numerical_columns:
    anomalies = detect_anomalies(important_df, col)
    print(f"Anomalies in {col}:")
    print(anomalies.shape[0])

Anomalies in loan_amount:
50
Anomalies in rate_of_interest:
0
Anomalies in Upfront_charges:
33
Anomalies in term:
0
Anomalies in property_value:
344
Anomalies in Credit_Score:
0
Anomalies in income:
484


In [10]:
def replace_anomalies_with_minmax_values(df, column):
    sorted_values = df[column].sort_values()
    Q1 = sorted_values.quantile(LOW_QUANTILE)
    Q3 = sorted_values.quantile(HIGH_QUANTILE)
    IQR = Q3 - Q1
    lower_bound = Q1 - IRQ_COEFF * IQR
    upper_bound = Q3 + IRQ_COEFF * IQR

    min_real_value = sorted_values[sorted_values >= lower_bound].min()
    max_real_value = sorted_values[sorted_values <= upper_bound].max()

    df.loc[df[column] < lower_bound, column] = min_real_value
    df.loc[df[column] > upper_bound, column] = max_real_value

numerical_columns = ['loan_amount', 'rate_of_interest', 'Upfront_charges', 'term', 'property_value', 'Credit_Score', 'income']
for col in numerical_columns:
    replace_anomalies_with_minmax_values(important_df, col)

print("Anomalies replaced with real values. Updated DataFrame:")
important_df.head(5)

Anomalies replaced with real values. Updated DataFrame:


,loan_amount,rate_of_interest,Upfront_charges,term,property_value,Credit_Score,income,open_credit,Neg_ammortization,interest_only,lump_sum_payment,age,Region,business_or_commercial,occupancy_type,Secured_by,Status
0,116500,3.99,2596.45,360.0,118000.0,758,1740.0,0,0,0,0,1,0,0,0,0,1
1,206500,3.99,2596.45,360.0,418000.0,552,4980.0,0,0,0,1,4,1,1,0,0,1
2,406500,4.56,595.00,360.0,508000.0,834,9480.0,0,1,0,0,2,0,0,0,0,0
3,456500,4.25,2596.45,360.0,658000.0,587,11880.0,0,0,0,0,3,1,0,0,0,0
4,696500,4.00,0.00,360.0,758000.0,602,10440.0,0,0,0,0,1,1,0,0,0,0


# Scaling

In [11]:
def scale_columns(df, columns):
    for col in columns:
        mean = df[col].mean()
        std = df[col].std()
        df[col] = (df[col] - mean) / std

numerical_columns = ['loan_amount', 'rate_of_interest', 'Upfront_charges', 'term', 'property_value', 'Credit_Score', 'income']
scale_columns(important_df, numerical_columns)

print("Numerical columns scaled using z-score normalization:")
important_df.head(5)

Numerical columns scaled using z-score normalization:


,loan_amount,rate_of_interest,Upfront_charges,term,property_value,Credit_Score,income,open_credit,Neg_ammortization,interest_only,lump_sum_payment,age,Region,business_or_commercial,occupancy_type,Secured_by,Status
0,-1.176138,-0.085756,-0.165124,0.425608,-1.159564,0.502356,-1.055208,0,0,0,0,1,0,0,0,0,1
1,-0.682592,-0.085756,-0.165124,0.425608,-0.218627,-1.275409,-0.378533,0,0,0,1,4,1,1,0,0,1
2,0.414178,1.081446,-0.884268,0.425608,0.063654,1.158230,0.561294,0,1,0,0,2,0,0,0,0,0
3,0.688371,0.446652,-0.165124,0.425608,0.534122,-0.973362,1.062536,0,0,0,0,3,1,0,0,0,0
4,2.004495,-0.065279,-1.098059,0.425608,0.847767,-0.843913,0.761791,0,0,0,0,1,1,0,0,0,0
